# Imports

In [289]:
# import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import jinja2
import os
import re

# Grab File

In [173]:
with open('./sample.xml', 'r') as f:
    data = f.read()

bs_data = BeautifulSoup(data, 'lxml')

In [174]:
bs_data

<?xml version="1.0" encoding="UTF-8"?><html><body><root schemalocation="https: //www.weareavp.com/nunncenter/ohms/ohms.xsd" xmlns="https: //www.weareavp.com/nunncenter/ohms" xmlns:="" xsi="http: //www.w3.org/2001/XMLSchema-instance" xsi:="">
<record dt="2021-07-13" id="00094187">
<version>5.4</version>
<date format="yyyy-mm-dd" value="2017-05-25"></date>
<date_nonpreferred_format>Unknown Date</date_nonpreferred_format>
<title>John Doe, Douglas County, 2017</title>
<accession></accession>
<duration>46: 56</duration>
<collection_id>12</collection_id>
<collection_name>interviews</collection_name>
<repository>historical society</repository>
<format>MP3</format>
<file_name>buzzbomb_from_pasadena_20170525.mp3</file_name>
<clip_format>audio</clip_format>
<title>Background and immigration story</title>
<keywords>Family;Native language;Russian language;World Relief</keywords>
<subjects>Rich Feynman as mentor</subjects>
<transcript>Gathering Our Voice

Interviewee:  John Doe

Interviewer:  Peter

In [175]:
transcript = bs_data.find("transcript")

In [244]:
type(transcript.text)

str

In [250]:
transcript.text

"Gathering Our Voice\n\nInterviewee:  John Doe\n\nInterviewer:  Peter Venkman (Egon Spengler also present) Date:  May 25, 2017\n\nTranscribed by Rev\n\nPeter Venkman: Lorem ipsum dolor sit amet, consectetur adipiscing elit. Duis a sem ut tellus efficitur ultrices. Aliquam varius nisi tincidunt ex ullamcorper euismod. Pellentesque id egestas purus. In vitae justo erat. Aenean bibendum convallis tortor et aliquam. Cras ac eros vel nulla lobortis pulvinar. Mauris suscipit nulla feugiat elit varius, eu tempor eros lobortis. \n\nJohn Doe: Mauris varius est at enim vehicula malesuada. Donec sed felis ac nibh rutrum pretium pharetra at eros. Maecenas a lacinia urna. Maecenas gravida rhoncus accumsan. Nulla et gravida velit, ut condimentum orci. Proin efficitur justo justo, non feugiat tortor interdum in. Sed sollicitudin interdum leo et eleifend. \n\nPeter Venkman: Suspendisse potenti. Morbi bibendum vitae ex sit amet pellentesque. Ut ultrices nulla ac malesuada interdum. Proin convallis conv

In [177]:
transcript.prettify

<bound method Tag.prettify of <transcript>Gathering Our Voice

Interviewee:  John Doe

Interviewer:  Peter Venkman (Egon Spengler also present) Date:  May 25, 2017

Transcribed by Rev

Peter Venkman: Lorem ipsum dolor sit amet, consectetur adipiscing elit. Duis a sem ut tellus efficitur ultrices. Aliquam varius nisi tincidunt ex ullamcorper euismod. Pellentesque id egestas purus. In vitae justo erat. Aenean bibendum convallis tortor et aliquam. Cras ac eros vel nulla lobortis pulvinar. Mauris suscipit nulla feugiat elit varius, eu tempor eros lobortis. 

John Doe: Mauris varius est at enim vehicula malesuada. Donec sed felis ac nibh rutrum pretium pharetra at eros. Maecenas a lacinia urna. Maecenas gravida rhoncus accumsan. Nulla et gravida velit, ut condimentum orci. Proin efficitur justo justo, non feugiat tortor interdum in. Sed sollicitudin interdum leo et eleifend. 

Peter Venkman: Suspendisse potenti. Morbi bibendum vitae ex sit amet pellentesque. Ut ultrices nulla ac malesuada i

In [246]:
#transcript_string = str(transcript.text)
transcript_lines = transcript.text.split('\n')
transcript_np = np.array(transcript_lines)

In [251]:
transcript_np

array(['Gathering Our Voice', '', 'Interviewee:  John Doe', '',
       'Interviewer:  Peter Venkman (Egon Spengler also present) Date:  May 25, 2017',
       '', 'Transcribed by Rev', '',
       'Peter Venkman: Lorem ipsum dolor sit amet, consectetur adipiscing elit. Duis a sem ut tellus efficitur ultrices. Aliquam varius nisi tincidunt ex ullamcorper euismod. Pellentesque id egestas purus. In vitae justo erat. Aenean bibendum convallis tortor et aliquam. Cras ac eros vel nulla lobortis pulvinar. Mauris suscipit nulla feugiat elit varius, eu tempor eros lobortis. ',
       '',
       'John Doe: Mauris varius est at enim vehicula malesuada. Donec sed felis ac nibh rutrum pretium pharetra at eros. Maecenas a lacinia urna. Maecenas gravida rhoncus accumsan. Nulla et gravida velit, ut condimentum orci. Proin efficitur justo justo, non feugiat tortor interdum in. Sed sollicitudin interdum leo et eleifend. ',
       '',
       'Peter Venkman: Suspendisse potenti. Morbi bibendum vitae ex sit 

In [253]:
for i in range(len(transcript_np)):
    print(i, transcript_np[i])

0 Gathering Our Voice
1 
2 Interviewee:  John Doe
3 
4 Interviewer:  Peter Venkman (Egon Spengler also present) Date:  May 25, 2017
5 
6 Transcribed by Rev
7 
8 Peter Venkman: Lorem ipsum dolor sit amet, consectetur adipiscing elit. Duis a sem ut tellus efficitur ultrices. Aliquam varius nisi tincidunt ex ullamcorper euismod. Pellentesque id egestas purus. In vitae justo erat. Aenean bibendum convallis tortor et aliquam. Cras ac eros vel nulla lobortis pulvinar. Mauris suscipit nulla feugiat elit varius, eu tempor eros lobortis. 
9 
10 John Doe: Mauris varius est at enim vehicula malesuada. Donec sed felis ac nibh rutrum pretium pharetra at eros. Maecenas a lacinia urna. Maecenas gravida rhoncus accumsan. Nulla et gravida velit, ut condimentum orci. Proin efficitur justo justo, non feugiat tortor interdum in. Sed sollicitudin interdum leo et eleifend. 
11 
12 Peter Venkman: Suspendisse potenti. Morbi bibendum vitae ex sit amet pellentesque. Ut ultrices nulla ac malesuada interdum. Proi

In [257]:
title = re.sub(r'(.*)$', r'<h1>\1</h1>', transcript_np[0])
title

'<h1>Gathering Our Voice</h1>'

In [249]:
print(f"<h1>{transcript_np[0]}</h1>")

<h1>Gathering Our Voice</h1>


In [181]:
cnt = 0
body = transcript_np[7:]

for i in range(len(body)):
    if not body[i].strip():
        body[i] = '<p></p>'
        print(body[i])
    else:
        if re.match('^[A-Z]{2}:', body[i]):
            body[i] = re.sub(r'^([A-Z ]{2}):(.*$)', r'<strong>\1</strong>:\2', body[i])
            print(body[i])
        elif re.match('^\w+ \w+:', body[i]):
            body[i] = re.sub(r'(^\w+ \w+):(.*$)', r'<strong>\1</strong>:\2', body[i])
            print(body[i])
        else:
            print(body[i])
    

<p></p>
<strong>Peter Venkman</strong>: Lorem ipsum dolor sit amet, consectetur adipiscing elit. Duis a sem ut tellus efficitur ultrices. Aliquam varius nisi tincidunt ex ullamcorper euismod. Pellentesque id egestas purus. In vitae justo erat. Aenean bibendum convallis tortor et aliquam. Cras ac eros vel nulla lobortis pulvinar. Mauris suscipit nulla feugiat elit varius, eu tempor eros lobortis. 
<p></p>
<strong>John Doe</strong>: Mauris varius est at enim vehicula malesuada. Donec sed felis ac nibh rutrum pretium pharetra at eros. Maecenas a lacinia urna. Maecenas gravida rhoncus accumsan. Nulla et gravida velit, ut condimentum orci. Proin efficitur justo justo, non feugiat tortor interdum in. Sed sollicitudin interdum leo et eleifend. 
<p></p>
<strong>Peter Venkman</strong>: Suspendisse potenti. Morbi bibendum vitae ex sit amet pellentesque. Ut ultrices nulla ac malesuada interdum. Proin convallis convallis magna sed semper. Nullam ut risus iaculis, consectetur dolor id, dapibus eros

In [182]:
for line in transcript_np[7:]:
    print(line)

<p></p>
<strong>Peter Venkman</strong>: Lorem ipsum dolor sit amet, consectetur adipiscing elit. Duis a sem ut tellus efficitur ultrices. Aliquam varius nisi tincidunt ex ullamcorper euismod. Pellentesque id egestas purus. In vitae justo erat. Aenean bibendum convallis tortor et aliquam. Cras ac eros vel nulla lobortis pulvinar. Mauris suscipit nulla feugiat elit varius, eu tempor eros lobortis. 
<p></p>
<strong>John Doe</strong>: Mauris varius est at enim vehicula malesuada. Donec sed felis ac nibh rutrum pretium pharetra at eros. Maecenas a lacinia urna. Maecenas gravida rhoncus accumsan. Nulla et gravida velit, ut condimentum orci. Proin efficitur justo justo, non feugiat tortor interdum in. Sed sollicitudin interdum leo et eleifend. 
<p></p>
<strong>Peter Venkman</strong>: Suspendisse potenti. Morbi bibendum vitae ex sit amet pellentesque. Ut ultrices nulla ac malesuada interdum. Proin convallis convallis magna sed semper. Nullam ut risus iaculis, consectetur dolor id, dapibus eros

# Convert to HTML

In [183]:

template = '''<h1>{{title}}</h1>
              <p></p>
              <h3>{{interview}}</h3>
              <p></p>
              <h4>{{date}}</h4>
              <p></p>
              <h4>{{transcribed}}</h4>
              <p></p>
              {% for line in body %}
                  {% if line|length %}
                      {{line}}
                  {% else %}
                      <p></p>
                  {% endif %}
              {% endfor %}'''
templater = Template(template)
templater.render(title=transcript_np[0],
                 interview=transcript_np[2],
                 date=transcript_np[4],
                 transcribed=transcript_np[6],
                 body=body)

"<h1>Gathering Our Voice</h1>\n              <p></p>\n              <h3>Interviewee:  John Doe</h3>\n              <p></p>\n              <h4>Interviewer:  Peter Venkman (Egon Spengler also present) Date:  May 25, 2017</h4>\n              <p></p>\n              <h4>Transcribed by Rev</h4>\n              <p></p>\n              \n                  \n                      <p></p>\n                  \n              \n                  \n                      <strong>Peter Venkman</strong>: Lorem ipsum dolor sit amet, consectetur adipiscing elit. Duis a sem ut tellus efficitur ultrices. Aliquam varius nisi tincidunt ex ullamcorper euismod. Pellentesque id egestas purus. In vitae justo erat. Aenean bibendum convallis tortor et aliquam. Cras ac eros vel nulla lobortis pulvinar. Mauris suscipit nulla feugiat elit varius, eu tempor eros lobortis. \n                  \n              \n                  \n                      <p></p>\n                  \n              \n                  \n     

In [258]:
with open('results.html', 'w') as f:
    f.write(templater.render(title=transcript_np[0],
                 interview=transcript_np[2],
                 date=transcript_np[4],
                 transcribed=transcript_np[6],
                 body=body))

# Convert to PDF

In [212]:
transcript_np2 = np.array(transcript_lines)
body2 = transcript_np2[7:]

for i in range(len(body2)):
    if not body2[i].strip():
        body2[i] = '.LP'
        print(body2[i])
    else:
        if re.match('^[A-Z]{2}:', body2[i]):
            body2[i] = re.sub(r'^([A-Z ]{2}):(.*$)', r'\\fB\1\\fR:\2', body2[i])
            print(body2[i])
        elif re.match('^\w+ \w+:', body2[i]):
            body2[i] = re.sub(r'(^\w+ \w+):(.*$)', r'\\fB\1\\fR:\2', body2[i])
            print(body2[i])
        else:
            print(body2[i])

.LP
\fBPeter Venkman\fR: Lorem ipsum dolor sit amet, consectetur adipiscing elit. Duis a sem ut tellus efficitur ultrices. Aliquam varius nisi tincidunt ex ullamcorper euismod. Pellentesque id egestas purus. In vitae justo erat. Aenean bibendum convallis tortor et aliquam. Cras ac eros vel nulla lobortis pulvinar. Mauris suscipit nulla feugiat elit varius, eu tempor eros lobortis. 
.LP
\fBJohn Doe\fR: Mauris varius est at enim vehicula malesuada. Donec sed felis ac nibh rutrum pretium pharetra at eros. Maecenas a lacinia urna. Maecenas gravida rhoncus accumsan. Nulla et gravida velit, ut condimentum orci. Proin efficitur justo justo, non feugiat tortor interdum in. Sed sollicitudin interdum leo et eleifend. 
.LP
\fBPeter Venkman\fR: Suspendisse potenti. Morbi bibendum vitae ex sit amet pellentesque. Ut ultrices nulla ac malesuada interdum. Proin convallis convallis magna sed semper. Nullam ut risus iaculis, consectetur dolor id, dapibus eros. In tempor velit in dictum euismod. Etiam fe

In [241]:
Template.trim_blocks = True
Template.lstrip_blocks = True


templatePDF = '''
.TL
{{title}}
.AU
{{interview}}
.AU
{{date}}
.AI
{{transcribed}}
.ds LF \\fIGOV Interviews\\fP
.ds CF CR - \*(DY 
.ds RF Page %
{%- for line in body %}
{%- if line|length %}
{{line}}
{%- else %}
.LP
{%- endif %}
{%- endfor %}'''
templater2 = Template(templatePDF)
templater2.render(title=transcript_np2[0],
                 interview=transcript_np2[2],
                 date=transcript_np2[4],
                 transcribed=transcript_np2[6],
                 body=body2)

"\n.TL\nGathering Our Voice\n.AU\nInterviewee:  John Doe\n.AU\nInterviewer:  Peter Venkman (Egon Spengler also present) Date:  May 25, 2017\n.AI\nTranscribed by Rev\n.ds LF \\fIGOV Interviews\\fP\n.ds CF CR - \\*(DY \n.ds RF Page %\n.LP\n\\fBPeter Venkman\\fR: Lorem ipsum dolor sit amet, consectetur adipiscing elit. Duis a sem ut tellus efficitur ultrices. Aliquam varius nisi tincidunt ex ullamcorper euismod. Pellentesque id egestas purus. In vitae justo erat. Aenean bibendum convallis tortor et aliquam. Cras ac eros vel nulla lobortis pulvinar. Mauris suscipit nulla feugiat elit varius, eu tempor eros lobortis. \n.LP\n\\fBJohn Doe\\fR: Mauris varius est at enim vehicula malesuada. Donec sed felis ac nibh rutrum pretium pharetra at eros. Maecenas a lacinia urna. Maecenas gravida rhoncus accumsan. Nulla et gravida velit, ut condimentum orci. Proin efficitur justo justo, non feugiat tortor interdum in. Sed sollicitudin interdum leo et eleifend. \n.LP\n\\fBPeter Venkman\\fR: Suspendisse p

In [242]:
with open('results.roff', 'w') as f:
    f.write(templater2.render(title=transcript_np2[0],
                 interview=transcript_np2[2],
                 date=transcript_np2[4],
                 transcribed=transcript_np2[6],
                 body=body2))

In [243]:
myCmd = 'groff -ms -Tps ./results.roff | ps2pdf14 - results.pdf'
os.system(myCmd)

0

# Excel

## Import Excel Writing Library and Create Workbook

In [430]:
import openpyxl
from openpyxl.styles import Alignment, colors, Color, Font, PatternFill
from openpyxl import Workbook

In [442]:
xlsx = "interview_wb.xlsx"
wb = Workbook()
ws = wb.active

## Get Some Stats from the XML File

In [443]:
stats = {}
stats['versionXML']    = bs_data.find("version").text
stats['titleXML']      = bs_data.find("title").text
stats['duration']      = bs_data.find("duration").text
stats['collection_id'] = bs_data.find("collection_id").text
stats['repository']    = bs_data.find("repository").text
stats['file_format']   = bs_data.find("format").text
stats['file_name']     = bs_data.find("file_name").text
stats['keywords']      = bs_data.find("keywords").text
stats['subjects']      = bs_data.find("subjects").text

In [444]:
stats

{'versionXML': '5.4',
 'titleXML': 'John Doe, Douglas County, 2017',
 'duration': '46: 56',
 'collection_id': '12',
 'repository': 'historical society',
 'file_format': 'MP3',
 'file_name': 'buzzbomb_from_pasadena_20170525.mp3',
 'keywords': 'Family;Native language;Russian language;World Relief',
 'subjects': 'Rich Feynman as mentor'}

## Add Title to Spreadsheet

In [445]:
A1 = ws['A1']
A1.value = "XML Interview Breakdown"
ws.merge_cells('A1:B1')
ws['A1'].alignment = Alignment(horizontal='center')

## Style Title

In [446]:
ft = Font(bold=True)              # Font color
fill = PatternFill("solid", fgColor="FDEDD9")     # Fill color

A1.font = ft
A1.fill = fill

## Add Data

In [447]:
# Column headings
A3 = ws["A3"]
A3.value  = "Key"
A3.fill = PatternFill("solid", fgColor="E4DCD2")
B3 = ws["B3"] 
B3.value = "Value"
B3.fill = PatternFill("solid", fgColor="E4DCD2")

for row, (key, value) in enumerate(stats.items(), start=4):
    ws [f"A{row}"] = key
    ws [f"B{row}"] = value

## Save Spreadsheet

In [448]:
wb.save(xlsx)